In [2]:
import torch
from torch import nn
from torchinfo import summary

![](./images/resnet/resnet_bottleneck.png)

In [3]:
class StandardBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channel, inner_channel, stride=1, projection=None):
        super().__init__()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channel, inner_channel, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(inner_channel),
            nn.ReLU(),
            nn.Conv2d(inner_channel, inner_channel*self.expansion, kernel_size=3, padding=1),
            nn.BatchNorm2d(inner_channel*self.expansion)
        )

        self.projection = projection
        self.relu = nn.ReLU()
    
    def forward(self, x):
        residual = self.residual(x)

        if self.projection is not None:
            shortcut = self.projection(x)
        else:
            shortcut = x
        
        output = self.relu(residual + shortcut)
        return output

In [4]:
class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channel, inner_channel, stride=1, projection=None):
        super().__init__()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channel, inner_channel, kernel_size=1),
            nn.BatchNorm2d(inner_channel),
            nn.ReLU(),
            # bottleneck을 3*3 conv 레이어에서 수행함. (not 첫번째 conv 레이어)
            nn.Conv2d(inner_channel, inner_channel, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(inner_channel),
            nn.ReLU(),
            nn.Conv2d(inner_channel, inner_channel*self.expansion, kernel_size=1),
            nn.BatchNorm2d(inner_channel*self.expansion)
        )

        self.projection = projection
        self.relu = nn.ReLU()
    
    def forward(self, x):
        residual = self.residual(x)

        if self.projection is not None:
            shortcut = self.projection(x)
        else:
            shortcut = x
        
        return self.relu(residual + shortcut)

### Input_Size : (3, 224, 224)
![](./images/resnet/resnet_diagram.png)
![](./images/resnet/resnet50_diagram.png)
![](./images/resnet/resnet_table.png)

In [12]:
class ResNet(nn.Module):
    def __init__(self, kind_of_block, block_num_list, num_classes=1000):
        super().__init__()

        self.in_channel = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(kind_of_block, 64, block_num_list[0], stride=1)
        self.layer2 = self.make_layer(kind_of_block, 128, block_num_list[1], stride=2)
        self.layer3 = self.make_layer(kind_of_block, 256, block_num_list[2], stride=2)
        self.layer4 = self.make_layer(kind_of_block, 512, block_num_list[3], stride=2)

        self.GlobalAvgPool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(self.in_channel, num_classes)

        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.kaiming_normal_(module.weight)
                nn.init.constant_(module.bias, 0.0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.GlobalAvgPool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)

        return x
    
    def make_layer(self, kind_of_block, inner_channel, num_of_block, stride):
        if stride != 1 or self.in_channel != inner_channel*kind_of_block.expansion:
            projection = nn.Sequential(
                nn.Conv2d(self.in_channel, inner_channel*kind_of_block.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(inner_channel*kind_of_block.expansion)
            )
        else:
            projection = None
        
        layers = []
        layers += [kind_of_block(self.in_channel, inner_channel, stride=stride, projection=projection)]

        self.in_channel = inner_channel*kind_of_block.expansion
        for _ in range(0, num_of_block-1):
            layers += [kind_of_block(self.in_channel, inner_channel)]
        
        return nn.Sequential(*layers)

In [13]:
def Resnet18(num_classes):
    return ResNet(StandardBlock, [2, 2, 2, 2], num_classes=num_classes)

def Resnet34(num_classes):
    return ResNet(StandardBlock, [3, 4, 6, 3], num_classes=num_classes)

def Resnet50(num_classes):
    return ResNet(BottleneckBlock, [3, 4, 6, 3], num_classes=num_classes)

def Resnet101(num_classes):
    return ResNet(BottleneckBlock, [3, 4, 23, 3], num_classes=num_classes)

def Resnet152(num_classes):
    return ResNet(BottleneckBlock, [3, 8, 36, 3], num_classes=num_classes)

In [14]:
model = Resnet18(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 64, 56, 56]           --
│    └─StandardBlock: 2-1                [2, 64, 56, 56]           --
│    │    └─Sequential: 3-1              [2, 64, 56, 56]           74,112
│    │    └─ReLU: 3-2                    [2, 64, 56, 56]           --
│    └─StandardBlock: 2-2                [2, 64, 56, 56]           --
│    │    └─Sequential: 3-3              [2, 64, 56, 56]           74,112
│    │    └─ReLU: 3-4                    [2, 64, 56, 56]           --
├─Sequential: 1-6                        [2, 128, 28, 28]          --
│  

In [15]:
model = Resnet34(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 64, 56, 56]           --
│    └─StandardBlock: 2-1                [2, 64, 56, 56]           --
│    │    └─Sequential: 3-1              [2, 64, 56, 56]           74,112
│    │    └─ReLU: 3-2                    [2, 64, 56, 56]           --
│    └─StandardBlock: 2-2                [2, 64, 56, 56]           --
│    │    └─Sequential: 3-3              [2, 64, 56, 56]           74,112
│    │    └─ReLU: 3-4                    [2, 64, 56, 56]           --
│    └─StandardBlock: 2-3                [2, 64, 56, 56]           --
│  

In [16]:
model = Resnet50(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-1              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-1              [2, 256, 56, 56]          58,496
│    │    └─Sequential: 3-2              [2, 256, 56, 56]          17,152
│    │    └─ReLU: 3-3                    [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-2              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-4              [2, 256, 56, 56]          70,784
│    │    └─ReLU: 3-5                    [2, 256, 56, 56]          --

In [17]:
model = Resnet101(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-1              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-1              [2, 256, 56, 56]          58,496
│    │    └─Sequential: 3-2              [2, 256, 56, 56]          17,152
│    │    └─ReLU: 3-3                    [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-2              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-4              [2, 256, 56, 56]          70,784
│    │    └─ReLU: 3-5                    [2, 256, 56, 56]          --

In [18]:
model = Resnet152(num_classes=1000)
summary(model, (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 1000]                 --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-1              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-1              [2, 256, 56, 56]          58,496
│    │    └─Sequential: 3-2              [2, 256, 56, 56]          17,152
│    │    └─ReLU: 3-3                    [2, 256, 56, 56]          --
│    └─BottleneckBlock: 2-2              [2, 256, 56, 56]          --
│    │    └─Sequential: 3-4              [2, 256, 56, 56]          70,784
│    │    └─ReLU: 3-5                    [2, 256, 56, 56]          --